In [45]:
import pdfplumber
import pandas as pd
import re
import json

---  
# PDFplumber

In [46]:
# Chemin vers votre fichier PDF
pdf_path = r"C:\Users\pierrontl\Documents\GitHub\Fraude\code_Tom\detect_montant\detect_montantenv\test_pdf\f323dceb7808_1729514526151.pdf"

# Liste pour stocker les données des caractères
data = []

## Information sur les caractères

In [47]:
# Ouvrir le fichier PDF
with pdfplumber.open(pdf_path) as pdf:
    first_page = pdf.pages[0]  # Prendre la première page
    for char in first_page.chars:  # Parcourir tous les caractères
        # Extraire les informations souhaitées
        char_info = {
            'text': char['text'],
            'fontname': char['fontname'], # La police du caractère 
            'size': char['size'],
            'x0': char['x0'],
            'y0': char['y0'],
            'x1': char['x1'],
            'y1': char['y1'],
            'width': char['width'],
            'height': char['height'],
            'page_number': char['page_number'],
            'stroking_color': char['stroking_color'],
            'non_stroking_color': char['non_stroking_color'],
            'upright': char['upright']
        }
        data.append(char_info)  # Ajouter les informations à la liste

## Création du dataframe 

In [48]:
# Créer un DataFrame pandas à partir de la liste
df_tout_montant = pd.DataFrame(data)
df_tout_montant.head()

,text,fontname,size,x0,y0,x1,y1,width,height,page_number,stroking_color,non_stroking_color,upright
0,F,BZZZZZ+Montserrat-Bold,14.0,186.466,764.824,195.412,778.824,8.946,14.0,1,None,"(0.050980392156862744, 0.13725490196078433, 0....",True
1,a,BZZZZZ+Montserrat-Bold,14.0,195.202,764.824,203.840,778.824,8.638,14.0,1,None,"(0.050980392156862744, 0.13725490196078433, 0....",True
2,c,BZZZZZ+Montserrat-Bold,14.0,203.924,764.824,212.198,778.824,8.274,14.0,1,None,"(0.050980392156862744, 0.13725490196078433, 0....",True
3,t,BZZZZZ+Montserrat-Bold,14.0,212.310,764.824,218.400,778.824,6.090,14.0,1,None,"(0.050980392156862744, 0.13725490196078433, 0....",True
4,u,BZZZZZ+Montserrat-Bold,14.0,218.400,764.824,228.018,778.824,9.618,14.0,1,None,"(0.050980392156862744, 0.13725490196078433, 0....",True


## Extraire les montants

In [49]:
montants = []
for page in pdf.pages:
    # Extraire le texte brut de la page
    texte = page.extract_text()
    if texte:
        # Le motif ci-dessous cherche des montants au format 123.45, $123.45, ou 123,45 €
        motifs = re.findall(r'(\d+[ ]?[,][ ]?\d{2} ?[€]?)', texte)
        # Ajouter les montants à la liste
        montants.extend(motifs)

# Afficher les montants trouvés
print("Montants extraits :", montants)

Montants extraits : ['100,00 ', '42,50 €', '0,00 €', '97,50 €', '97,50 €', '55,00 €', '42,50 €', '0,00 €']


le regex doit prendre les montants : 

100 000 avec espace  
a virgule  
avec devise  
avec le point



puis ensuite lister tout les montants capter par le regex  
pour les faires correler avec les montants qui sont présents dans le tableau  

puis par la suite essayer de trouver qu'elle sont les montant total, net, taxes ...

# Comment détecter des montants dans un tableau 

In [50]:
# %pip install jpype1
# %pip install tabula-py

In [51]:
import tabula

## Détecter des tableaux

In [52]:
# Chemin vers votre fichier PDF
pdf_file = r"C:\Users\pierrontl\Documents\GitHub\Fraude\code_Tom\detect_montant\detect_montantenv\test_pdf\f323dceb7808_1729514526469.pdf"
output_json = r"C:\Users\pierrontl\Documents\GitHub\Fraude\code_Tom\detect_montant\detect_montantenv\plumber\output.json"
# Extraire les tableaux de toutes les pages
tables = tabula.read_pdf(pdf_file, pages='all', multiple_tables=True)


# Sauvegarder les tableaux en json
tabula.convert_into(pdf_file, r"C:\Users\pierrontl\Documents\GitHub\Fraude\code_Tom\detect_montant\detect_montantenv\plumber\output.json", output_format="json", pages='all')

In [53]:
# Ouvrir et charger le fichier JSON
with open(output_json, 'r', encoding='utf-8') as f:
    data_json = json.load(f)

## Extraire les données du tableau

In [54]:
# Liste pour stocker les lignes traitées
rows = []

# Vérifier si 'data_json' contient des données
if len(data_json) > 0:
    for row in data_json[0]['data']:
        # Pour chaque cellule, extraire le texte et traiter les \r\r\r ou \r\r\r\r comme des sauts de ligne
        processed_row = []
        for cell in row:
            # Diviser le texte lorsqu'on rencontre \r\r\r ou plus
            split_content = re.split(r'\r{3,}', cell['text'])
            # Ajouter le contenu traité à la ligne
            processed_row.append(split_content)
        
        # Ajouter la ligne traitée à la liste des lignes
        rows.append(processed_row)

    # À ce stade, `rows` contient des listes de listes (car chaque cellule peut avoir plusieurs parties)
    # On va aplatir les données pour créer un nouveau DataFrame en traitant chaque cellule divisée comme une nouvelle ligne

    # Trouver le nombre maximum de sous-lignes dans les cellules pour garantir l'alignement
    max_sublines_per_cell = max(max(len(cell) for cell in row) for row in rows)

    # Créer une liste pour le nouveau DataFrame
    new_rows = []

    # Parcourir les lignes et ajuster le nombre de sous-lignes
    for row in rows:
        for i in range(max_sublines_per_cell):
            # On crée une nouvelle ligne pour chaque "sous-ligne"
            new_row = []
            for cell in row:
                if i < len(cell):
                    new_row.append(cell[i])  # Prendre la sous-ligne correspondante
                else:
                    new_row.append('')  # Remplir par une chaîne vide si pas assez de sous-lignes
            new_rows.append(new_row)

    # Créer le DataFrame à partir des nouvelles lignes
    df = pd.DataFrame(new_rows)

    print("Tableau détecté et chargé dans un DataFrame avec saut de ligne pris en compte.")
    print(df)
else:
    print("Aucun tableau détecté.")

Aucun tableau détecté.


In [55]:
df

,0,1,2
0,Payé le 03/10/2024 par Chèque,TOTAL,"97,50 €"


---  
## Essaie avec easyOCR et tabula

In [56]:
import fitz, os 
from mylib import paths
from typing import Tuple
import shutil

def pdf2img(pdfFile: str, pages: Tuple = None):
    # On charge le document
    pdf = fitz.open(pdfFile)
    # On détermine la liste des fichiers générés
    pngFiles = []
    # Pour chaque page du pdf
    pngPath = str(paths.rootPath_img) + str(paths.tmpDirImg) + os.path.basename(str(pdfFile).split('.')[0])
    
    try:
        for pageId in range(pdf.page_count):
            if str(pages) != str(None):
                if str(pageId) not in str(pages):
                    continue

            # On récupère la page courante
            page = pdf[pageId]
            # On convertit la page courante
            pageMatrix = fitz.Matrix(2, 2)
            pagePix = page.get_pixmap(matrix=pageMatrix, alpha=False)
            # On exporte la page générée

            # Si le répertoire dédié au pdf n'existe pas encore, on le crée
            if not os.path.exists(pngPath):
                os.makedirs(pngPath)

            pngFile = pngPath + "_" + f"page{pageId+1}.png"
            pagePix.save(pngFile)
            pngFiles.append(pngFile)

        pdf.close()

        # On retourne la liste des pngs générés
        return pngFiles

    finally:
        # On supprime le répertoire et son contenu après le traitement
        if os.path.exists(pngPath):
            shutil.rmtree(pngPath)

In [57]:
# import easyocr
# from pdf2image import convert_from_path
# import re
# from mylib import functions
 
# # Initialiser EasyOCR avec le support du français
# reader = easyocr.Reader(['fr'], gpu=False)  # Si vous avez un GPU, vous pouvez mettre gpu=True
 
# # Fonction pour convertir un PDF en images et extraire le texte avec EasyOCR
# def pdf_to_text_with_easyocr(pdf_path):
#     images = pdf2img(pdf_path)
#     full_text = ""
#     # Parcourir chaque page du PDF
#     for img in images:
#         # Extraire le texte avec EasyOCR
#         result = reader.readtext(img, detail=0)  # detail=0 renvoie uniquement le texte
#         full_text += " ".join(result) + "\n"
#     return full_text
 
# # Fonction pour extraire des désignations ou informations structurées
# def extract_information(text):
#     sentences = text.split('\n')  # Diviser le texte en phrases par lignes
#     relevant_sentences = []
#     # Parcourir chaque phrase pour trouver celles qui sont pertinentes
#     for sentence in sentences:
#         # Utiliser des heuristiques pour détecter les phrases avec des désignations
#         # Exemple : phrases contenant des noms propres ou des mentions numériques (âges, montants, etc.)
#         if re.search(r'\d{1,2}\s?(ans|€|jours)', sentence) or re.search(r'\bM\.|Mme\b', sentence):
#             relevant_sentences.append(sentence)
#     return relevant_sentences

 
# # Extraire le texte du PDF avec EasyOCR
# text = pdf_to_text_with_easyocr(pdf_file)

# # Extraire les informations pertinentes
# relevant_information = extract_information(text)
 
# # Afficher les désignations détectées
# if relevant_information:
#     print("Informations pertinentes détectées :")
#     for info in relevant_information:
#         print(info)
# else:
#     print("Aucune information pertinente détectée.")